In [1]:
using LowLevelFEM
import LowLevelFEM as FEM
gmsh.initialize()

Info    : Increasing process stack size (8192 kB < 16 MB)


In [2]:
gmsh.open("body1-2D.geo")

Info    : Reading 'body1-2D.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 30%] Meshing curve 2 (Line)
Info    : [ 60%] Meshing curve 3 (Line)
Info    : [ 80%] Meshing curve 4 (Line)
Info    : Done meshing 1D (Wall 0.00056861s, CPU 0.000567s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.00783317s, CPU 0.007817s)
Info    : Meshing order 2 (curvilinear on)...
Info    : [  0%] Meshing curve 1 order 2
Info    : [ 30%] Meshing curve 2 order 2
Info    : [ 50%] Meshing curve 3 order 2
Info    : [ 70%] Meshing curve 4 order 2
Info    : [ 90%] Meshing surface 1 order 2
Info    : Surface mesh: worst distortion = 1 (0 elements in ]0, 0.2]); worst gamma = 0.781163
Info    : Done meshing order 2 (Wall 0.000923179s, CPU 0.000921s)
Info    : 357 nodes 198 elements
Info    : Done reading 'body1-2D.geo'


In [3]:
mat = FEM.material("body")
problem = FEM.Problem([mat], type=:AxiSymmetricHeatConduction)

Info    : RCMK renumbering...
Info    : Done RCMK renumbering (bandwidth is now 61)


LowLevelFEM.Problem("body1-2D", :AxiSymmetricHeatConduction, 2, 1, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666)], 1.0, 357)

In [4]:
supp = FEM.temperatureConstraint("left", T=0)
load = FEM.heatFlux("right", qn=-1)

("right", 1, 0, 0)

In [5]:
T0 = FEM.initialTemperature(problem, "body", T=0)

LowLevelFEM.ScalarField(Matrix{Float64}[], [0.0; 0.0; … ; 0.0; 0.0;;], [0.0], Int64[], 1, :T, LowLevelFEM.Problem("body1-2D", :AxiSymmetricHeatConduction, 2, 1, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666)], 1.0, 357))

In [6]:
FEM.initialTemperature!("left", T0, T=0)
T0

LowLevelFEM.ScalarField(Matrix{Float64}[], [0.0; 0.0; … ; 0.0; 0.0;;], [0.0], Int64[], 1, :T, LowLevelFEM.Problem("body1-2D", :AxiSymmetricHeatConduction, 2, 1, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666)], 1.0, 357))

In [7]:
K = FEM.heatConductionMatrix(problem)
C = FEM.heatCapacityMatrix(problem)
q = FEM.heatFluxVector(problem, [load])

LowLevelFEM.ScalarField(Matrix{Float64}[], [0.0; 0.1308996938995745; … ; 0.0; 0.0;;], Float64[], Int64[], 1, :qn, LowLevelFEM.Problem("body1-2D", :AxiSymmetricHeatConduction, 2, 1, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666)], 1.0, 357))

In [8]:
FEM.applyBoundaryConditions!(K, C, q, [supp])

In [9]:
λmax = FEM.largestEigenValue(K, C)

161562.79141166466

In [10]:
ϑ = 0.33
T = FEM.FDM(K, C, q, T0, 200 * (2 / ((1 - 2ϑ) * λmax)), 10 * 2 / ((1 - 2ϑ) * λmax), ϑ=ϑ)

LowLevelFEM.ScalarField(Matrix{Float64}[], [0.0 0.0 … 0.0 0.0; 0.0 0.002644830444842868 … -0.8667328493325244 1.4552714235166495; … ; 0.0 2.0907265732800836e-6 … -0.05986754181062643 0.13312668815817752; 0.0 4.349831195705991e-6 … 0.06948173008849448 -0.08656855572588229], [0.0, 0.00036409082127011156, 0.0007281816425402231, 0.0010922724638103348, 0.0014563632850804462, 0.0018204541063505577, 0.002184544927620669, 0.0025486357488907806, 0.002912726570160892, 0.0032768173914310035, 0.003640908212701115, 0.004004999033971226, 0.004369089855241338, 0.00473318067651145, 0.005097271497781562, 0.005461362319051674, 0.005825453140321786, 0.006189543961591898, 0.00655363478286201, 0.0069177256041321215], Int64[], 20, :T, LowLevelFEM.Problem("body1-2D", :AxiSymmetricHeatConduction, 2, 1, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666)], 1.0, 357))

In [11]:
ϑ = 0.66
T = FEM.FDM(K, C, q, T0, 0.15, 0.01, ϑ=ϑ)

LowLevelFEM.ScalarField(Matrix{Float64}[], [0.0 0.0 … 0.0 0.0; 0.0 0.012121530046179921 … 0.06540158786374889 0.06840790461983615; … ; 0.0 0.005276551637608011 … 0.05831544416510987 0.06124834953421595; 0.0 0.00570802450428725 … 0.05904444980835037 0.06199196309294858], [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.060000000000000005, 0.07, 0.08, 0.09, 0.09999999999999999, 0.10999999999999999, 0.11999999999999998, 0.12999999999999998, 0.13999999999999999], Int64[], 15, :T, LowLevelFEM.Problem("body1-2D", :AxiSymmetricHeatConduction, 2, 1, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666)], 1.0, 357))

In [12]:
FEM.showDoFResults(T, :T, visible=true)

1

In [13]:
q1 = FEM.solveHeatFlux(T)

LowLevelFEM.VectorField([[0.0 -0.7378240105496102 … -1.025693324301331 -1.0414608980229474; 0.0 -0.0011129965148889975 … 0.00030193798761235513 0.00032434667751601864; … ; 0.0 -0.9117353098320614 … -1.0133148658954534 -1.0245255708764034; 0.0 -0.00045505024536297844 … 0.0005471045078875303 0.0005848745906089636], [0.0 -1.0860352038863887 … -1.000999395625008 -1.0076542030550257; 0.0 0.0007947705814120853 … 0.0008882092071238468 0.0009428199164585038; … ; 0.0 -0.912030621722692 … -1.013299345263169 -1.0245079621662683; 0.0 -5.256251290469693e-6 … 0.0005234647988072716 0.000558054498671634], [0.0 -1.0458527507951125 … -1.0027659885373286 -1.0103166609734449; 0.0 -0.0001402941440262727 … 0.0006864738575984575 0.000726756273097956; … ; 0.0 -0.8820833096183798 … -1.0160853614731664 -1.0281164359788337; 0.0 0.0009631377078497122 … 0.0010856779282686357 0.0011497899077639318], [0.0 -0.19807986971391423 … -4.196609760463232 -4.421429445030323; 0.0 0.01482037780762585 … 0.36258815671512457 0.38

In [14]:
FEM.probe(q1, 0.5, 0.5, 0, step=10)

2-element Vector{Float64}:
 -1.0404450218969667
  0.0020624831301164644

In [15]:
FEM.showHeatFluxResults(q1, :qvec)

Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)


2

In [16]:
q2 = FEM.solveHeatFlux(T, DoFResults=true)

LowLevelFEM.VectorField(Matrix{Float64}[], [0.0 -1.1326265559023114 … -28.886331411411753 -30.459591900261834; 0.0 -0.11832727642773351 … -2.986485213769326 -3.149013430773377; … ; 0.0 -0.651909998557965 … -1.04732919626435 -1.0671121606837013; 0.0 0.002375305075317455 … 0.0017128343656588885 0.0018085362762801371], [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.060000000000000005, 0.07, 0.08, 0.09, 0.09999999999999999, 0.10999999999999999, 0.11999999999999998, 0.12999999999999998, 0.13999999999999999], Int64[], 15, :q2D, LowLevelFEM.Problem("body1-2D", :AxiSymmetricHeatConduction, 2, 1, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666)], 1.0, 357))

In [17]:
FEM.showDoFResults(q2, :qvec)

3

In [18]:
FEM.probe(q2, 0.5, 0.5, 0, step=10)

2-element Vector{Float64}:
 -1.0404930265575518
  0.0023792349006771757

In [19]:
q3 = FEM.elementsToNodes(q1)

LowLevelFEM.VectorField(Matrix{Float64}[], [0.0 -1.1326265559023114 … -28.886331411411753 -30.459591900261834; 0.0 -0.11832727642773351 … -2.986485213769326 -3.149013430773377; … ; 0.0 -0.651909998557965 … -1.04732919626435 -1.0671121606837013; 0.0 0.002375305075317455 … 0.0017128343656588885 0.0018085362762801371], [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.060000000000000005, 0.07, 0.08, 0.09, 0.09999999999999999, 0.10999999999999999, 0.11999999999999998, 0.12999999999999998, 0.13999999999999999], Int64[], 15, :q2D, LowLevelFEM.Problem("body1-2D", :AxiSymmetricHeatConduction, 2, 1, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666)], 1.0, 357))

In [20]:
FEM.showDoFResults(q3, :qvec)

4

In [21]:
err1 = FEM.fieldError(q1)

LowLevelFEM.VectorField(Matrix{Float64}[], [0.0 0.11832727642773344 … 2.9864852137693205 3.1490134307733744; 0.0 0.11832727642773351 … 2.986485213769326 3.149013430773377; … ; 0.0 0.00023879051160785592 … 0.0002710010623833137 0.0002850349771360561; 0.0 0.0002102775886964059 … 0.00023864201951800945 0.00025100020634688525], [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.060000000000000005, 0.07, 0.08, 0.09, 0.09999999999999999, 0.10999999999999999, 0.11999999999999998, 0.12999999999999998, 0.13999999999999999], Int64[], 15, :q2D, LowLevelFEM.Problem("body1-2D", :AxiSymmetricHeatConduction, 2, 1, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666)], 1.0, 357))

In [22]:
FEM.showDoFResults(err1, :vector)

5

In [23]:
FEM.openPostProcessor()

-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XOpenIM() failed
XRequest.18: BadValue 0x0


In [24]:
gmsh.finalize()